在前一节中，我们讨论了使用树的一般节点对普通树的深度优先遍历。其中，树节点对象包括两个字段，分别是数据域和子节点集合。由于一般树的子节点可能拥有任意多个子节点，因此使用该方式定义树节点对象会使树结构的处理变的复杂，本节讨论如何使用二叉树结构表示一般树，在此基础上，讨论二叉树表示形式下，一般树的遍历方法。

在二叉树结构中，每一个树结点最多包括两个子节点，称之为左节点和右节点。二叉树具有结构简单，易于处理的优点。为了使用二叉树结构表示一般树，我们定义如下形式树节点：

In [1]:
case class BinaryNode[T](data: T, leftNode: Option[BinaryNode[T]],
                         rightNode: Option[BinaryNode[T]])

defined class BinaryNode

每一树节点包括三个字段，分别为数据域，左节点和右节点，其中左右节点均可能为空。当使用二叉树表示一般树结构时，左节点执行当前节点的第一个子节点，右节点指向当前节点紧邻的第一个兄弟节点。因此，可以将普通的一般树转化为如下二叉树形式。

建立该树所对应的二叉树节点：

In [2]:
val node1 = BinaryNode[Char]('M', None, None)
val node2 = BinaryNode[Char]('L', None, Option(node1))
val node3 = BinaryNode[Char]('K', None, Option(node2))
val node4 = BinaryNode[Char]('H', None, None)
val node5 = BinaryNode[Char]('G', Option(node3), Option(node4))
val node6 = BinaryNode[Char]('C', Option(node5), None)
val node7 = BinaryNode[Char]('J', None, None)
val node8 = BinaryNode[Char]('I', None, None)
val node9 = BinaryNode[Char]('F', None, None)
val node10 = BinaryNode[Char]('E', Option(node7), Option(node9))
val node11 = BinaryNode[Char]('D', Option(node8), Option(node10))
val node12 = BinaryNode[Char]('B', Option(node11), Option(node6))
val node13 = BinaryNode[Char]('A', Option(node12), None)

node1: BinaryNode[Char] = BinaryNode('M', None, None)
node2: BinaryNode[Char] = BinaryNode(
  'L',
  None,
  Some(BinaryNode('M', None, None))
)
node3: BinaryNode[Char] = BinaryNode(
  'K',
  None,
  Some(BinaryNode('L', None, Some(BinaryNode('M', None, None))))
)
node4: BinaryNode[Char] = BinaryNode('H', None, None)
node5: BinaryNode[Char] = BinaryNode(
  'G',
  Some(
    BinaryNode(
      'K',
      None,
      Some(BinaryNode('L', None, Some(BinaryNode('M', None, None))))
    )
  ),
  Some(BinaryNode('H', None, None))
)
node6: BinaryNode[Char] = BinaryNode(
  'C',
  Some(
    BinaryNode(
      'G',
      Some(
        BinaryNode(
          'K',
          None,
          Some(BinaryNode('L', None, Some(BinaryNode('M', None, None))))
        )
      ),
      Some(BinaryNode('H', None, None))
    )
  ),
  None
)
node7: BinaryNode[Char] = BinaryNode('J', None, None)
node8: BinaryNode[Char] = BinaryNode('I', None, None)
node9: BinaryNode[Char] = BinaryNode('F', None, None)
node10: Binary

## 1. 深度优先遍历

### 1.1 前序遍历

一般树的前序遍历与其二叉树形式的前序遍历顺序一致，此处使用迭代方法对二叉树进行前序遍历：

In [7]:
def preOrder[T, V](node: Option[BinaryNode[T]], fun: T => V): List[V] = node match {
  case None => Nil
  case x =>
    fun(x.get.data) :: List(x.get.leftNode, x.get.rightNode).flatMap(n => preOrder(n, fun))
}

val res = preOrder[Char, Char](Option(node13), x => x)
res.foldLeft("start:")(_ + "->" + _.toString)

defined function preOrder
res: List[Char] = List(
  'A',
  'B',
  'D',
  'I',
  'E',
  'J',
  'F',
  'C',
  'G',
  'K',
  'L',
  'M',
  'H'
)
res6_2: String = "start:->A->B->D->I->E->J->F->C->G->K->L->M->H"

### 1.3 后序遍历

一般树的后序遍历与其对应二叉树的中序遍历相一致，此处使用迭代方式对二叉树进行中序遍历：

In [8]:
def postOrder[T, V](node: Option[BinaryNode[T]], fun: T => V): List[V] = node match {
  case None => Nil
  case x =>
    postOrder(x.get.leftNode, fun) ++ List(fun(x.get.data)) ++ postOrder(x.get.rightNode, fun)
}

val res = postOrder[Char, Char](Option(node13), x => x)
res.foldLeft("start:")(_ + "->" + _.toString)

defined function postOrder
res: List[Char] = List(
  'I',
  'D',
  'J',
  'E',
  'F',
  'B',
  'K',
  'L',
  'M',
  'G',
  'H',
  'C',
  'A'
)
res7_2: String = "start:->I->D->J->E->F->B->K->L->M->G->H->C->A"

## 2. 广度优先遍历

树的广度优先遍历也称之为层序遍历，使用队列结构可以很容易实现在二叉树上实现一般树的层层序遍历：

In [9]:
import collection.mutable
def breadthFirst[T, V](node: Option[BinaryNode[T]],
                       fun: T => V): List[V] = {
  val res = mutable.ListBuffer[V]()
  val queue = mutable.Queue[Option[BinaryNode[T]]]()
  if (node.nonEmpty){
    queue.enqueue(node)
    while(queue.nonEmpty) {
      val node = queue.dequeue()
      res += fun(node.get.data)
      // 入队该节点的头节点，和头节点的所有兄弟节点
        
      if (node.get.leftNode.nonEmpty) {
        var leftNode = node.get.leftNode
        queue.enqueue(leftNode) // 入队头节点
        while (leftNode.get.rightNode.nonEmpty) { // 入队兄弟节点
          queue.enqueue(leftNode.get.rightNode)
          leftNode = leftNode.get.rightNode
        }
      }
    }
    res.toList
  }
  else Nil
}

val res = breadthFirst[Char, Char](Option(node13), x => x)
res.foldLeft("start:")(_ + "->" + _.toString)

import collection.mutable

defined function breadthFirst
res: List[Char] = List(
  'A',
  'B',
  'C',
  'D',
  'E',
  'F',
  'G',
  'H',
  'I',
  'J',
  'K',
  'L',
  'M'
)
res8_3: String = "start:->A->B->C->D->E->F->G->H->I->J->K->L->M"